In [1]:
%load_ext sql

In [2]:
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import sqlalchemy
import boto3

## Import Configuration For Redshift Cluster

In [3]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')

DB_NAME= config.get("CLUSTER","DB_NAME")
DB_USER= config.get("CLUSTER","DB_USER")
DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DB_PORT = config.get("CLUSTER","DB_PORT")
DB_ENDPOINT = config.get("CLUSTER","HOST")
DB_ROLE_ARN = config.get("IAM_ROLE","ARN")
DB_REGION = config.get("CLUSTER","DB_REGION")
DB_IAM_ROLE_NAME=config.get("CLUSTER","DB_IAM_ROLE_NAME")
DB_CLUSTER_IDENTIFIER = config.get("CLUSTER","DB_CLUSTER_IDENTIFIER")

## Connect To Redshift Cluster

In [4]:
connect_to_redshift = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DB_ENDPOINT, DB_PORT, DB_NAME)
%sql $connect_to_redshift

## Number of Records For Each Table 

In [ ]:
se = %sql SELECT COUNT(*) FROM staging_events;
ss = %sql SELECT COUNT(*) FROM staging_songs;
songplays = %sql SELECT COUNT(*) FROM songplays;
users = %sql SELECT COUNT(*) FROM users;
songs = %sql SELECT COUNT(*) FROM songs;
artists = %sql SELECT COUNT(*) FROM artists;
time = %sql SELECT COUNT(*) FROM time;
num_rows = [item[0][0] for item in [se, ss, songplays, users, songs, artists, time]]

In [ ]:
se = %sql SELECT COUNT(*) FROM staging_events;
ss = %sql SELECT COUNT(*) FROM staging_songs;
songplays = %sql SELECT COUNT(DISTINCT songplay_id) FROM songplays;
users = %sql SELECT COUNT(DISTINCT user_id) FROM users;
songs = %sql SELECT COUNT(DISTINCT song_id) FROM songs;
artists = %sql SELECT COUNT(DISTINCT artist_id) FROM artists;
time = %sql SELECT COUNT(DISTINCT start_time) FROM time;
uniq_pk = [item[0][0] for item in [se, ss, songplays, users, songs, artists, time]]

In [7]:
dwh = pd.DataFrame(index = ['staging_events', 'staging_songs', 'songplays', 'users', 'songs', 'artists', 'time'],
                   data = {'num_rows': num_rows, 'uniq_pk': uniq_pk})

In [8]:
dwh

,num_rows,uniq_pk
staging_events,8056,8056
staging_songs,14896,14896
songplays,333,333
users,96,96
songs,14896,14896
artists,10025,9553
time,8023,8023


## Number of Nulls Per Table

** Songplays **

In [ ]:
null_pk =        %sql SELECT COUNT(songplay_id) FROM songplays WHERE songplay_id IS NULL;
null_userId =    %sql SELECT COUNT(user_id)     FROM songplays WHERE user_id IS NULL;
null_st =        %sql SELECT COUNT(start_time)  FROM songplays WHERE start_time IS NULL;
null_level =     %sql SELECT COUNT(level)       FROM songplays WHERE level IS NULL;
null_songId =    %sql SELECT COUNT(song_id)     FROM songplays WHERE song_id IS NULL;
null_artistId =  %sql SELECT COUNT(artist_id)   FROM songplays WHERE artist_id IS NULL;
null_sessionId = %sql SELECT COUNT(session_id)  FROM songplays WHERE session_id IS NULL;
null_location =  %sql SELECT COUNT(location)    FROM songplays WHERE location IS NULL;
null_userAgent = %sql SELECT COUNT(user_agent)  FROM songplays WHERE user_agent IS NULL;
null_sp = [item[0][0] for item in [null_pk, null_userId, null_st, null_level, null_songId, null_artistId, null_sessionId, null_location, null_userAgent]]
null_songplays = pd.DataFrame(data = {'num_nulls': null_sp}, index = ['songplay_id', 'user_id', 'start_time', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent'])

In [10]:
null_songplays

,num_nulls
songplay_id,0
user_id,0
start_time,0
level,0
song_id,0
artist_id,0
session_id,0
location,0
user_agent,0


** Users **

In [ ]:
null_userId = %sql SELECT COUNT(user_id)    FROM users WHERE user_id IS NULL;
null_fn =     %sql SELECT COUNT(first_name) FROM users WHERE first_name IS NULL;
null_ln =     %sql SELECT COUNT(last_name)  FROM users WHERE last_name IS NULL;
null_gender = %sql SELECT COUNT(gender)     FROM users WHERE gender IS NULL;
null_level =  %sql SELECT COUNT(level)      FROM users WHERE level IS NULL;
null_u = [item[0][0] for item in [null_userId, null_fn, null_ln, null_gender, null_level]]
null_users = pd.DataFrame(data = {'num_nulls': null_u}, index = ['user_id', 'first_name', 'last_name', 'gender', 'level'])

In [12]:
null_users

,num_nulls
user_id,0
first_name,0
last_name,0
gender,0
level,0


** Songs **

In [ ]:
null_userId =   %sql SELECT COUNT(song_id)   FROM songs WHERE song_id IS NULL;
null_title =    %sql SELECT COUNT(title)     FROM songs WHERE title IS NULL;
null_artistId = %sql SELECT COUNT(artist_id) FROM songs WHERE artist_id IS NULL;
null_year =     %sql SELECT COUNT(year)      FROM songs WHERE year IS NULL;
null_duration = %sql SELECT COUNT(duration)  FROM songs WHERE duration IS NULL;
null_s = [item[0][0] for item in [null_userId, null_title, null_artistId, null_year, null_duration]]
null_songs = pd.DataFrame(data = {'num_nulls': null_s}, index = ['user_id', 'title', 'artist_id', 'year', 'duration'])

In [14]:
null_songs

,num_nulls
user_id,0
title,0
artist_id,0
year,0
duration,0


** Artists **

In [ ]:
null_artistId = %sql SELECT COUNT(artist_id) FROM artists WHERE artist_id IS NULL;
null_name =     %sql SELECT COUNT(name)      FROM artists WHERE name IS NULL;
null_location = %sql SELECT COUNT(location)  FROM artists WHERE location IS NULL;
null_lat =      %sql SELECT COUNT(latitude)  FROM artists WHERE latitude IS NULL;
null_long =     %sql SELECT COUNT(longitude) FROM artists WHERE longitude IS NULL;
null_a = [item[0][0] for item in [null_artistId, null_name, null_location, null_lat, null_long]]
null_artists = pd.DataFrame(data = {'num_nulls': null_a}, index = ['artist_id', 'name', 'location', 'latitude', 'longitude'])

In [16]:
null_artists

,num_nulls
artist_id,0
name,0
location,0
latitude,0
longitude,0


## Example records Per Table 

** Songplays **

In [ ]:
response = %sql SELECT * FROM songplays LIMIT 5;
response = response.DataFrame()

In [18]:
response

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,2,2018-11-28 22:56:08.796,73,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,954,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
1,18,2018-11-28 22:32:42.796,73,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,954,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,34,2018-11-22 01:59:04.796,15,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,50,2018-11-19 06:45:01.796,24,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,66,2018-11-25 16:14:24.796,49,paid,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,923,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...


** Users **

In [ ]:
response = %sql SELECT * FROM users LIMIT 5;
response = response.DataFrame()

In [20]:
response

,user_id,first_name,last_name,gender,level
0,39,Walter,Frye,M,free
1,18,Jacob,Rogers,M,free
2,72,Hayden,Brock,F,paid
3,97,Kate,Harrell,F,paid
4,67,Colm,Santana,M,free


** Songs **

In [ ]:
response = %sql SELECT * FROM songs LIMIT 5;
response = response.DataFrame()

In [22]:
response

,song_id,title,artist_id,year,duration
0,SOVCPPL12A8C13F177,Toca Toca,ARWB73A1187FB5C9DF,2007,185.024849
1,SOLOZLZ12A58A78D88,Indian Carnival,ARU63BJ1187FB3E997,1954,126.327710
2,SOQDNVA12AF729B1D1,Duck Down,ARH7M7Z1187B99B894,0,169.873830
3,SOUPGMR12AB01820DD,Døden Nær,AR1ILHX1187FB53892,2005,266.631379
4,SOXUEGX12A8C13C161,Operation: M.O.V.E.,AR8JO2B1187B98EBB6,2004,625.057509


** Artistis **

In [ ]:
response = %sql SELECT * FROM artists LIMIT 5;
response = response.DataFrame()

In [24]:
response

,artist_id,name,location,latitude,longitude
0,ARWUK9L11C8A42C37B,Workhorse,Mississippi,32.585070,-89.873739
1,ARAQJZS1187B9B55E9,Musetta,ITALY,None,None
2,ARPKDY81187B98DFF7,Har Mar Superstar,,None,None
3,AR9Q34X1187B9A28B6,Caedmon's Call,,None,None
4,AR8UA7Z1187B9AD9C9,Étienne Daho,Rennes,48.107509,-1.684469


** Time **

In [ ]:
response = %sql SELECT * FROM time LIMIT 5;
response = response.DataFrame()

In [26]:
response

,start_time,hour,day,week,month,year,weekday
0,2018-11-13 01:12:29.796,1,13,46,11,2018,2
1,2018-11-13 08:20:35.796,8,13,46,11,2018,2
2,2018-11-13 08:25:59.796,8,13,46,11,2018,2
3,2018-11-13 08:53:21.796,8,13,46,11,2018,2
4,2018-11-13 09:02:33.796,9,13,46,11,2018,2


## Check max and min for numerical attribute

In [ ]:
response = %sql SELECT max(duration), min(duration) FROM songs;
response = response.DataFrame()

In [28]:
response

,max,min
0,2709.237099,6.373420


## Check level in users table was properly updated

In [ ]:
response = %sql SELECT user_id FROM (SELECT user_id, level FROM songplays GROUP BY user_id, level) AS u GROUP BY user_id HAVING COUNT(*) > 1 ORDER BY user_id;
ids = tuple([item[0] for item in response])

In [ ]:
response  = %sql SELECT user_id, level, dat FROM (SELECT user_id, level, DATE(start_time) AS dat FROM songplays WHERE user_id in $ids) GROUP BY user_id, level, dat ORDER BY user_id, dat; 

In [33]:
response = response.DataFrame()
response

,user_id,level,dat
0,16,free,2018-11-11
1,16,paid,2018-11-14
2,16,paid,2018-11-29
3,16,paid,2018-11-30
4,36,free,2018-11-09
5,36,paid,2018-11-09
6,36,paid,2018-11-10
7,36,paid,2018-11-16
8,36,paid,2018-11-26
9,36,paid,2018-11-27


** Current level for the users **

In [ ]:
response = %sql SELECT user_id, level FROM users WHERE user_id in $ids ORDER BY user_id;
response = response.DataFrame()

In [35]:
response

,user_id,level
0,16,paid
1,36,paid
2,49,paid
3,88,paid


## Clean up resources

In [45]:
redshift = boto3.client('redshift',region_name=DB_REGION,aws_access_key_id=KEY,aws_secret_access_key=SECRET)
iam = boto3.client('iam',region_name=DB_REGION,aws_access_key_id=KEY,aws_secret_access_key=SECRET)

In [46]:
response = redshift.delete_cluster(ClusterIdentifier=DB_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)

In [47]:
waiter = redshift.get_waiter('cluster_deleted')
waiter.wait(ClusterIdentifier=DB_CLUSTER_IDENTIFIER)

In [ ]:
iam.detach_role_policy(RoleName=DB_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DB_IAM_ROLE_NAME)  